<a href="https://colab.research.google.com/github/Shreya-07/CMPE297_Assignment2/blob/main/Assignment2_partc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# link  prediction of graph

**By: Shreya Nimbhorkar**

## Installing all the required libraries

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


1.12.1+cu113
     |████████████████████████████████| 7.9 MB 17.3 MB/s 
     |████████████████████████████████| 3.5 MB 14.6 MB/s 


In [2]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE

## Loading Dataset

In [4]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [5]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [6]:
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


## Model Training

###Initialization of Auto Encoder

Training with new negative samples for each training epoch

Validation/Test evaluation with roc_auc_score (Area Under the Receiver Operating Characteristic Curve)

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [8]:
out_channels = 2
num_features = dataset.num_features
epochs = 100

model = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

In [10]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [11]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    if epoch % 10 == 0:
      print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 010, AUC: 0.6775, AP: 0.7139
Epoch: 020, AUC: 0.6832, AP: 0.7357
Epoch: 030, AUC: 0.6822, AP: 0.7456
Epoch: 040, AUC: 0.7293, AP: 0.7663
Epoch: 050, AUC: 0.7902, AP: 0.7999
Epoch: 060, AUC: 0.7966, AP: 0.8045
Epoch: 070, AUC: 0.8000, AP: 0.8055
Epoch: 080, AUC: 0.8034, AP: 0.8072
Epoch: 090, AUC: 0.8051, AP: 0.8067
Epoch: 100, AUC: 0.8093, AP: 0.8099


## Evaluation

In [12]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.4429, -0.3630],
        [-0.9332,  0.7933],
        [ 0.7404, -0.6667],
        ...,
        [-0.3692,  0.3173],
        [ 0.7060, -0.6181],
        [ 0.8707, -0.8504]], device='cuda:0', grad_fn=<AddBackward0>)